In [1]:

#!python3 -m pip install pytorch_lightning
#!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from model import LSTM_Model
from pytorch_lightning import seed_everything, LightningModule, Trainer
from torch import save
from pytorch_lightning.callbacks import EarlyStopping

torch.manual_seed(1)

Dataset version: 0.0


In [ ]:
seed_everything(42)
device = 'cuda' if torch.cuda.is_available() else 'cpu' #Check for cuda 
#device = 'cpu'
print(f'Using {device} device')

early_stop_callback = EarlyStopping(monitor='val_loss', min_delta=0.00, patience=5, verbose=True, mode='min')

model = LSTM_Model('data/speaker_rec/').to(device)
#trainer = Trainer(max_epochs=100, min_epochs=1, auto_lr_find=False, auto_scale_batch_size=False, callbacks=[early_stop_callback],enable_checkpointing=False)
trainer = Trainer(max_epochs=3, min_epochs=1, auto_lr_find=False, auto_scale_batch_size=False,enable_checkpointing=False)
trainer.tune(model)

trainer.fit(model)
#save(model.state_dict(), '/trained_model')

Global seed set to 42


Using cuda device


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name         | Type             | Params
--------------------------------------------------
0 | lstm         | LSTM             | 42.0 K
1 | fc           | Linear           | 5.5 M 
2 | flatten      | Flatten          | 0     
3 | output       | Sigmoid          | 0     
4 | sm           | Softmax          | 0     
5 | loss         | CrossEntropyLoss | 0     
6 | accuracy     | Accuracy         | 0     
7 | val_accuracy | Accuracy         | 0     
--------------------------------------------------
5.6 M     Trainable params
0         Non-trainable params
5.6 M     Total params
22.311    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

{'progress_bar': {'val_loss': 1.609655499458313}, 'log': {'val_loss': 1.609655499458313}, 'val_loss': 1.609655499458313}
Accuracy: 0.19203628599643707


Training: 0it [00:00, ?it/s]